In [7]:
import luigi
import pandas as pd
import os
import logging



In [8]:
class ReadAllData(luigi.Task):
    def run(self):
        df_delinquance = pd.read_csv("./CleanData/delinquance.csv", sep=",")
        df_canton = pd.read_csv("./CleanData/canton.csv", sep=",")

        display(df_delinquance)
        display(df_canton)

    def output(self):
        return luigi.LocalTarget('./CleanData/data.csv')

In [9]:

config = luigi.configuration.get_config()
config.set('core', 'no_lock', 'False')


luigi.build([ReadAllData()], local_scheduler=False, no_lock=True)

DEBUG: Checking if ReadAllData() is complete
INFO: Informed scheduler that task   ReadAllData__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 9896] Worker Worker(salt=7410879189, workers=1, host=DESKTOP-PBH195U, username=Marin, pid=9896) running   ReadAllData()


,classe,annee,Code.département,Code.région,unité.de.compte,millPOP,millLOG,faits,POP,LOG,tauxpourmille
0,Coups et blessures volontaires,17,01,84,victime,17,17,1464,643350,"313061,4522700008","2,27558871531825568"
1,Coups et blessures volontaires,17,02,32,victime,17,17,1866,534490,"265548,3196165975","3,49117850661378126"
2,Coups et blessures volontaires,17,03,84,victime,17,17,1013,337988,"207724,6632561111","2,99714782773352928"
3,Coups et blessures volontaires,17,04,93,victime,17,17,521,163915,"127940,4230924362","3,17847664948296371"
4,Coups et blessures volontaires,17,05,93,victime,17,17,381,141284,"135492,6309531483","2,69669601653407343"
...,...,...,...,...,...,...,...,...,...,...,...
1207,Destructions et dégradations volontaires,17,971,1,infraction,17,17,2871,390253,"227525,3489634245","7,35676599539273202"
1208,Destructions et dégradations volontaires,17,972,2,infraction,17,17,2309,372594,"212577,1492133937","6,19709388771692460"
1209,Destructions et dégradations volontaires,17,973,3,infraction,17,17,1596,268700,"88986,4502167438","5,93970971343505827"
1210,Destructions et dégradations volontaires,17,974,4,infraction,17,17,5359,853659,"369894,5213297095","6,27768230640103386"


,Code du departement,Libelle du departement,Code du canton,Libelle du canton,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,...,N°Panneau,Sexe,Nom,Prenom,Voix,% Voix/Ins,% Voix/Exp,Tendance,Code Tendance,Parti polititque
0,1,Ain,1,Amberieu-en-Bugey,19922,3909,"19,62",16013,"80,38",353,...,2,F,LE PEN,Marine,4408,"22,13","28,37",d'extrême droite,1,RN
1,1,Ain,1,Amberieu-en-Bugey,19922,3909,"19,62",16013,"80,38",353,...,9,M,MeLENCHON,Jean-Luc,3196,"16,04","20,57",d'extrême gauche,2,LFI
2,1,Ain,1,Amberieu-en-Bugey,19922,3909,"19,62",16013,"80,38",353,...,3,M,MACRON,Emmanuel,3125,"15,69","20,11",gauche,3,LREM
3,1,Ain,1,Amberieu-en-Bugey,19922,3909,"19,62",16013,"80,38",353,...,11,M,FILLON,Francois,2483,"12,46","15,98",droite,4,LR
4,1,Ain,1,Amberieu-en-Bugey,19922,3909,"19,62",16013,"80,38",353,...,1,M,DUPONT-AIGNAN,Nicolas,883,"4,43","5,68",droite,4,DLF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20895,ZZ,Francais etablis hors de France,1,Canton fictif,1264113,704383,"55,72",559730,"44,28",3169,...,1,M,DUPONT-AIGNAN,Nicolas,8837,"0,7","1,59",droite,4,DLF
20896,ZZ,Francais etablis hors de France,1,Canton fictif,1264113,704383,"55,72",559730,"44,28",3169,...,10,M,ASSELINEAU,Francois,5578,"0,44","1,01",droite,4,UPR
20897,ZZ,Francais etablis hors de France,1,Canton fictif,1264113,704383,"55,72",559730,"44,28",3169,...,6,M,POUTOU,Philippe,3414,"0,27","0,62",d'extrême gauche,2,NPA
20898,ZZ,Francais etablis hors de France,1,Canton fictif,1264113,704383,"55,72",559730,"44,28",3169,...,8,M,LASSALLE,Jean,2530,"0,2","0,46",droite,4,RES


INFO: [pid 9896] Worker Worker(salt=7410879189, workers=1, host=DESKTOP-PBH195U, username=Marin, pid=9896) done      ReadAllData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ReadAllData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=7410879189, workers=1, host=DESKTOP-PBH195U, username=Marin, pid=9896) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 ran successfully:
    - 1 ReadAllData()

This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====



True